In [1]:
from utils.data_util import *
import pandas as pd
import numpy as np
from glob import glob


In [2]:
# def dcm_to_train_set(db = "original_data/archive/MITOS_WSI_CCMCT_ODAEL_train_dcm.sqlite", source_dir = "original_data/archive",\
#                          dest_dir = "datasets/new", tile_size = 512, cell_size = 40):
yolo_dcm_to_train_set_includeNone(tile_size=320)

1 / 21 066c94c4c161224077a9 : 0 / 47850


c:\CCMCT\utils\data_util.py:238: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  local_cells = cells[['x','y','annoid']][cells['x'] > location[0]][cells['y'] > location[1]]\


1 / 21 066c94c4c161224077a9 : 500 / 47850
1 / 21 066c94c4c161224077a9 : 1000 / 47850
1 / 21 066c94c4c161224077a9 : 1500 / 47850
1 / 21 066c94c4c161224077a9 : 2000 / 47850
1 / 21 066c94c4c161224077a9 : 2500 / 47850
1 / 21 066c94c4c161224077a9 : 3000 / 47850
1 / 21 066c94c4c161224077a9 : 3500 / 47850
1 / 21 066c94c4c161224077a9 : 4000 / 47850
1 / 21 066c94c4c161224077a9 : 4500 / 47850
1 / 21 066c94c4c161224077a9 : 5000 / 47850
1 / 21 066c94c4c161224077a9 : 5500 / 47850
1 / 21 066c94c4c161224077a9 : 6000 / 47850
1 / 21 066c94c4c161224077a9 : 6500 / 47850
1 / 21 066c94c4c161224077a9 : 7000 / 47850
1 / 21 066c94c4c161224077a9 : 7500 / 47850
1 / 21 066c94c4c161224077a9 : 8000 / 47850
1 / 21 066c94c4c161224077a9 : 8500 / 47850
1 / 21 066c94c4c161224077a9 : 9000 / 47850
1 / 21 066c94c4c161224077a9 : 9500 / 47850
1 / 21 066c94c4c161224077a9 : 10000 / 47850
1 / 21 066c94c4c161224077a9 : 10500 / 47850
1 / 21 066c94c4c161224077a9 : 11000 / 47850
1 / 21 066c94c4c161224077a9 : 11500 / 47850
1 / 21 0

In [3]:
# train / valid split
from sklearn.model_selection import train_test_split

files = glob("datasets/img_320/labels/*.txt")
file_lists = []
for file_name in files:
    file_lists.append(file_name.split("\\")[1].split(".")[0])

train_files, test_files = train_test_split(file_lists, random_state=7, test_size=0.2)
print(len(train_files), len(test_files))

with open("datasets/img_320/train.txt", 'w') as f:
    for train_file in train_files:
        f.write(train_file+"\n")

with open("datasets/img_320/val.txt", 'w') as f:
    for test_file in test_files:
        f.write(test_file+"\n")

993156 248289


In [4]:
# train test split 칸에서 작성된 txt파일에 따라 이미지, 라벨 다른 폴더로 분할
import shutil

source_dir = "datasets/img_320"

img_dest = "datasets/Normal/images"
label_dest = "datasets/Normal/labels"

if not os.path.exists(img_dest):
    os.makedirs(img_dest)
    os.makedirs(img_dest + "/train")
    os.makedirs(img_dest + "/val")

if not os.path.exists(label_dest):
    os.makedirs(label_dest)
    os.makedirs(label_dest + "/train")
    os.makedirs(label_dest + "/val")

train_txt = "datasets/img_320/train.txt"
val_txt = "datasets/img_320/val.txt"

with open(train_txt, 'r') as f:
    train_files = f.readlines()

with open(val_txt, 'r') as f:
    val_files = f.readlines()    

for train_file in train_files:
    temp = train_file.strip()
    shutil.move(source_dir + "/images/"+temp+".png", img_dest + "/train")
    shutil.move(source_dir + "/labels/"+temp+".txt", label_dest + "/train")

for val_file in val_files:
    temp = val_file.strip() 
    shutil.move(source_dir + "/images/"+temp+".png", img_dest + "/val")
    shutil.move(source_dir + "/labels/"+temp+".txt", label_dest + "/val")


In [4]:
cells = get_all_cells("datasets/new/labels")
cells['label'].value_counts()

3    39515
4    36563
1    30851
2    19621
0    12569
Name: label, dtype: int64

In [ ]:
ds = ReadableDicomDataset('datasets/archive/fff27b79894fe0157b08.dcm')
# location=(69700,17100)
# size=(500,500)

location=(60000,40000)
size=(3000,3000)
img = Image.fromarray(ds.read_region(location=location,size=size))
img

In [ ]:
# Get the annotation coordinates, offset by the left upper coordinate (location)
# NOTE: We would actually have to check the label class - which we omit for the sake of simplicity here
DB = sqlite3.connect('datasets/archive/MITOS_WSI_CCMCT_ODAEL_train_dcm.sqlite')
cur = DB.cursor()

cells = cur.execute(f"""SELECT coordinateX-{location[0]}, coordinateY-{location[1]}
                        from Annotations_coordinates where slide==7 and 
                        coordinateX>{location[0]} and coordinateX<{location[0]+size[0]} and 
                        coordinateY>{location[1]} and coordinateY<{location[1]+size[1]}""").fetchall()

from PIL import ImageDraw
draw = ImageDraw.Draw(img)
for (cx,cy) in cells:
    r=25
    draw.ellipse([(cx-r,cy-r),(cx+r,cy+r)],outline=(255,0,0,255))
img

In [15]:
import os
with open("temp.txt", 'w+') as f:
    a=1
print(os.path.getsize("temp.txt"))

0
